In [1]:
%pip install -q langchain langchain_community text_generation

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 아래에서 계속 사용할 LLM을 만들어줍니다. 

import warnings

from langchain_community.llms import HuggingFaceTextGenInference

warnings.filterwarnings("ignore")

# 야놀자에서 만든 EEVE Korean 8B 모델을 사용합니다.
llm = HuggingFaceTextGenInference(
    inference_server_url="http://eeve-korean-instruct-10-8b-tgi.serving.70-220-152-1.sslip.io"
)

In [3]:
# CustomEmbedding (SDS임베딩)을 사용합니다. 
# 로컬에서 사용할 때는 import 해주세요. 

from typing import (List)

import requests
from langchain_core.embeddings import Embeddings


class CustomEmbedding(Embeddings):
    def __init__(self):
        self.embed_url = 'http://sds-embed.serving.70-220-152-1.sslip.io/v1/models/embed:predict'

    def call_embed(self, url, texts):
        data = {
            "instances": texts
        }
        response = requests.post(url=url, json=data)
        result = response.json()
        return result['predictions']

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        주어진 텍스트를 임베딩하여 벡터로 반환 합니다.
        """
        embed_list = self.call_embed(url=self.embed_url, texts=texts)
        return embed_list

    def embed_query(self, text: str) -> List[float]:
        """Embed query text."""

        embed_list = self.call_embed(url=self.embed_url, texts=[text])
        return embed_list[0]

In [15]:
%pip install -qU unstructured unstructured-inference pillow_heif pikepdf lark 

ERROR: Could not find a version that satisfies the requirement onnxruntime>=1.17.0 (from unstructured-inference) (from versions: 1.7.0, 1.8.0, 1.8.1, 1.9.0, 1.10.0, 1.11.0, 1.11.1, 1.12.0, 1.12.1, 1.13.1, 1.14.0, 1.14.1, 1.15.0, 1.15.1, 1.16.0, 1.16.1, 1.16.2, 1.16.3)
ERROR: No matching distribution found for onnxruntime>=1.17.0 (from unstructured-inference)
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: http://nexus.sdsdev.co.kr:8081/repository/pypi/simple
     |████████████████████████████████| 27.0 MB 23.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS

# 텍스트를 600자 단위로 분할
text_splitter = CharacterTextSplitter(chunk_size=600, chunk_overlap=0)

# TextLoader 를 통해 텍스트 파일을 로드
split_docs = TextLoader(
    "../data/appendix-keywords.txt").load_and_split(text_splitter)

# FAISS 를 통해 벡터 저장소 생성
faiss_db = FAISS.from_documents(split_docs, CustomEmbedding())

# retriever 생성
retriever = faiss_db.as_retriever()

# similarity_search 를 통해 유사도 높은 1개 문서를 검색
relevant_docs = retriever.get_relevant_documents("Page Rank 에 대하여 알려줘")

print(f"문서의 개수: {len(relevant_docs)}")
print("[검색 결과]\n")
print(relevant_docs[0].page_content)

문서의 개수: 4
[검색 결과]

정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를 찾는 과정입니다. 이는 대부분의 검색 엔진과 데이터베이스 시스템에서 기본적인 검색 방식으로 사용됩니다.
예시: 사용자가 "커피숍 서울"이라고 검색하면, 관련된 커피숍 목록을 반환합니다.
연관키워드: 검색 엔진, 데이터 검색, 정보 검색

Page Rank

정의: 페이지 랭크는 웹 페이지의 중요도를 평가하는 알고리즘으로, 주로 검색 엔진 결과의 순위를 결정하는 데 사용됩니다. 이는 웹 페이지 간의 링크 구조를 분석하여 평가합니다.
예시: 구글 검색 엔진은 페이지 랭크 알고리즘을 사용하여 검색 결과의 순위를 정합니다.
연관키워드: 검색 엔진 최적화, 웹 분석, 링크 분석

데이터 마이닝

정의: 데이터 마이닝은 대량의 데이터에서 유용한 정보를 발굴하는 과정입니다. 이는 통계, 머신러닝, 패턴 인식 등의 기술을 활용합니다.
예시: 소매업체가 고객 구매 데이터를 분석하여 판매 전략을 수립하는 것은 데이터 마이닝의 예입니다.
연관키워드: 빅데이터, 패턴 인식, 예측 분석

멀티모달 (Multimodal)


/home/user/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


In [35]:
# Multi-Query Retriever 예제

# Build a sample vectorDB
from langchain.text_splitter import (
    CharacterTextSplitter,
)
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.faiss import FAISS

# Load blog post
loader = TextLoader("../data/appendix-keywords.txt")
data = loader.load()

# Split
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)

# VectorDB
embedding = CustomEmbedding()
vectordb = FAISS.from_documents(documents=splits, embedding=embedding)

In [36]:
from langchain.retrievers.multi_query import MultiQueryRetriever

question = "Word2Vec 에 관한 내용을 알려줘!"

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

In [37]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [38]:
unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['Alternative Question 1: Word2Vec의 핵심 개념과 그 응용에 대해 설명해줄 수 있나요?', '', 'Alternative Question 2: Word2Vec의 알고리즘 원리와 성능 향상을 위한 팁에 대해 논의해 주실 수 있나요?', '', 'Alternative Question 3: Word2Vec과 다른 단어 임베딩 모델들의 비교 분석과 강점, 약점에 대해 설명해 줄 수 있나요?']


8

In [39]:
for i in range(len(unique_docs)):
    print(f"문서 {i+1}:\n{unique_docs[i].page_content}\n")
    print("===" * 20)

문서 1:
정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)

문서 2:
Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token

문서 3:
정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.
예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.
연관키워드: 토큰화, 자연어 처리, 구문 분석

Tokenizer

정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.
예시: "I l

In [6]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

QA_PROMPT = PromptTemplate(
    input_variables=["query", "contexts"],
    template="""You are a helpful assistant who answers user queries using the
    contexts provided. If the question cannot be answered using the information
    provided say "I don't know".

    Contexts:
    {contexts}

    Question: {query}""",
)

# Chain
qa_chain = LLMChain(llm=llm, prompt=QA_PROMPT)

/home/user/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [7]:
# MultiQuery 기반으로 QA Chain을 실행한 결과

out = qa_chain(
    inputs={
        "query": question,
        "contexts": "\n---\n".join([d.page_content for d in unique_docs])
    }
)
out["text"]

NameError: name 'question' is not defined

### Contextual Compression 

![Contextual Compression](../docs/contextual_compression.png)

In [42]:
# Contextual compression

# Helper function for printing docs
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


In [44]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS

documents = TextLoader("../data/appendix-keywords.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
retriever = FAISS.from_documents(texts,
                                 CustomEmbedding()
                                 ).as_retriever()

docs = retriever.get_relevant_documents("Word2Vec에 대해 알려줘")
pretty_print_docs(docs)

Document 1:

정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.
예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석, 텍스트 생성 등의 작업을 수행할 수 있습니다.
연관키워드: 자연어 처리, 딥러닝, 라이브러리

Digital Transformation

정의: 디지털 변환은 기술을 활용하여 기업의 서비스, 문화, 운영을 혁신하는 과정입니다. 이는 비즈니스 모델을 개선하고 디지털 기술을 통해 경쟁력을 높이는 데 중점을 둡니다.
예시: 기업이 클라우드 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.
연관키워드: 혁신, 기술, 비즈니스 모델

Crawling

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)

정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.
예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.
연관키워드: 자연어 처리, 딥러닝, 텍스트 생성

FAISS (Facebook AI Similarity Search)

In [45]:
## LLMChainExtractor를 이용하여 Contextual Compression 추가

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(llm)

# it needs a base retriever (we're using FAISS Retriever) and a compressor (Made above)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor,
                                                       base_retriever=retriever)

# compressor prompt
compressor.llm_chain.prompt

compressed_docs = compression_retriever.get_relevant_documents("Word2Vec에 대해 알려줘")

pretty_print_docs(compressed_docs)

Document 1:

* Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
* Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
* 연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
----------------------------------------------------------------------------------------------------
Document 2:

* 임베딩
* 벡터화
* 딥러닝
* Token
* 토큰화
* 자연어 처리
* 구문 분석
* VectorStore
* 벡터 형식으로 변환된 데이터 저장
* 검색, 분류 및 기타 데이터 분석 작업 사용
----------------------------------------------------------------------------------------------------
Document 3:

- Word2Vec에 대해 알려줘
- 정의: Word2Vec은 단어 벡터와 그 관계를 학습하는 데 사용되는 딥러닝 모델입니다.
예시: Word2Vec은 두 단어 사이의 유사성을 판단하기 위해 단어 벡터 간의 거리를 측정할 수 있습니다.
연관키워드: 자연어 처리, 텍스트 분석, 딥러닝
----------------------------------------------------------------------------------------------------
Document 4:

- 인공신경망
- 머신러닝
- 데이터 분석
- 데이터베이스
- 데이터 모델링
- 데이터 관리
- DataFrame
- 판다스
- 데이터 분석
- 파이썬
- 데이터 처리
- Attention 메커니즘
- 딥러닝
- 자연어 처리
- 시퀀스 모델링

No_Output:
신뢰도: 100%


In [46]:
# LLMChainFilter는 강력하지만 간단한 압축기로, 검색 문서를 필터링하는 Chain

from langchain.retrievers.document_compressors import LLMChainFilter

_filter = LLMChainFilter.from_llm(llm)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents("Word2Vec에 대해 알려줘")

pretty_print_docs(compressed_docs)

Document 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.
예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.
연관키워드: 토큰화, 자연어 처리, 구문 분석

Tokenizer

정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.
예시: "I love programming."이라는 문장을 ["I", "love", "programming", "."]으로 분할합니다.
연관키워드: 토큰화, 자연어 처리, 구문 분석

VectorStore

정의: 벡터스토어는 벡터 형식으로 변환된 데이터를 저장하는 시스템입니다. 이는 검색, 분류 및 기타 데이터 분석 작업에 사용됩니다.
예시: 단어 임베딩 벡터들을 데이터베이스에 저장하여 빠르게 접근할 수 있습니다.
연관키워드: 임베딩, 데이터베이스, 벡터화

SQL


In [47]:
# Embedding Filter
# 임베딩 필터는 문서와 쿼리를 임베딩하고 쿼리와 충분히 유사한 임베딩이 있는 
# 문서만 반환함으로써 더 저렴하고 빠른 옵션을 제공

from langchain.retrievers.document_compressors import EmbeddingsFilter

embeddings = CustomEmbedding()
embeddings_filter = EmbeddingsFilter( embeddings=embeddings, similarity_threshold=0.76)
compression_retriever = ContextualCompressionRetriever(base_compressor=embeddings_filter, base_retriever=retriever)

compressed_docs = compression_retriever.get_relevant_documents("Word2Vec에 대해 알려줘")

pretty_print_docs(compressed_docs)

Document 1:

정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.
예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석, 텍스트 생성 등의 작업을 수행할 수 있습니다.
연관키워드: 자연어 처리, 딥러닝, 라이브러리

Digital Transformation

정의: 디지털 변환은 기술을 활용하여 기업의 서비스, 문화, 운영을 혁신하는 과정입니다. 이는 비즈니스 모델을 개선하고 디지털 기술을 통해 경쟁력을 높이는 데 중점을 둡니다.
예시: 기업이 클라우드 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.
연관키워드: 혁신, 기술, 비즈니스 모델

Crawling

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)

정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.
예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.
연관키워드: 자연어 처리, 딥러닝, 텍스트 생성

FAISS (Facebook AI Similarity Search)

In [48]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

QA_PROMPT = PromptTemplate(
    input_variables=["query", "contexts"],
    template="""You are a helpful assistant who answers user queries using the
    contexts provided. If the question cannot be answered using the information
    provided say "I don't know".

    Contexts:
    {contexts}

    Question: {query}""",
)

# Chain
qa_chain = LLMChain(llm=llm, prompt=QA_PROMPT)

In [49]:
out = qa_chain(
    inputs={
        "query": question,
        "contexts": "\n---\n".join([d.page_content for d in compressed_docs])
    }
)

print(out["text"])



답변: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다. Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다. Word2Vec은 단어 임베딩을 생성하는 데 사용되며, 이는 다양한 자연어 이해 및 생성 작업에 사용됩니다.


In [51]:
# 압축기와 문서 변환기를 함께 묶기
# DocumentCompressorPipeline을 사용하면 여러 여러 항목을 쉽게 결합
# BaseDocumentTransformers - 문서에서 변환을 수행할 수 있습니다. 텍스트를 분할하고
# EmbeddingsRedundantFilter - 분할 또는 변환 후 관련되지 않은 항목을 필터링합니다.

from langchain.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=0, separator=". ")

# EmbeddingsRedundantFilter 인스턴스를 생성합니다. 이 필터는 임베딩을 기반으로 중복 문서를 필터링합니다.
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)

# EmbeddingsFilter 인스턴스를 생성합니다. 이 필터는 0.76의 유사도 임계값을 사용하여 관련성이 높은 문서를 필터링합니다.
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)

# DocumentCompressorPipeline 인스턴스를 생성하여 문서 압축 파이프라인을 구성합니다.
# 이 파이프라인은 앞서 정의한 스플리터와 두 필터 (중복 필터와 관련성 필터)를 포함합니다.
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[splitter, redundant_filter, relevant_filter]
)

In [52]:
compression_retriever = ContextualCompressionRetriever(base_compressor=pipeline_compressor,
                                                       base_retriever=retriever)

compressed_docs = compression_retriever.get_relevant_documents("Word2Vec에 대해 알려줘")

pretty_print_docs(compressed_docs)

Document 1:

이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.
예시: "I love programming."이라는 문장을 ["I", "love", "programming", "."]으로 분할합니다.
연관키워드: 토큰화, 자연어 처리, 구문 분석

VectorStore

정의: 벡터스토어는 벡터 형식으로 변환된 데이터를 저장하는 시스템입니다. 이는 검색, 분류 및 기타 데이터 분석 작업에 사용됩니다.
예시: 단어 임베딩 벡터들을 데이터베이스에 저장하여 빠르게 접근할 수 있습니다.
연관키워드: 임베딩, 데이터베이스, 벡터화

SQL
----------------------------------------------------------------------------------------------------
Document 2:

이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)

정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.
예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.
연관키워드: 자연어 처리, 딥러닝, 텍스트 생성

FAISS (Facebook AI Similarity Search)
----------------------------------------------------------------------------------------------------
Document 3:

정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.
예시: HuggingFace의 Transformers 라이

In [53]:
### different pipeline

## making the pipeline
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[splitter, compressor, redundant_filter, relevant_filter]
)

In [54]:
compression_retriever = ContextualCompressionRetriever(base_compressor=pipeline_compressor,
                                                       base_retriever=retriever)

compressed_docs = compression_retriever.get_relevant_documents("Word2Vec에 대해 알려줘")

pretty_print_docs(compressed_docs)

Document 1:

Given the following context, the relevant parts for the question "What is Word2Vec?" are:

* Word2Vec is primarily used on sequential data (e.g., text, time series data).
* Example: In translation models, the Attention mechanism focuses more on important parts of the input sentence to create accurate translations.

Note: Since the context does not provide a direct definition or explanation of Word2Vec, the relevant parts are extracted based on the context's mention of Word2Vec and its usage in translation models.
----------------------------------------------------------------------------------------------------
Document 2:

Word2Vec에 대해 알려줘
이 맥락에서, Word2Vec은 언급되지 않았습니다.

그러므로 NO_OUTPUT이 답입니다.
----------------------------------------------------------------------------------------------------
Document 3:

> Context:
>>>
이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이

<br>

### Ensemble Retriever

앙상블 리트리버는 다양한 리트리버 알고리즘을 조합해, get_relevant_documents() 메서드 결과를 조합(앙상블)해 결과의 재순위를 매기는 역할을 하며, 이를 통해 단일 알고리즘보다 더 나은 성능을 달성할 수 있습니다.

가장 일반적인 패턴은 BM25와 같은 희소(Sparse) 리트리버와 임베딩 유사도 같은 밀집(Dense) 리트리버를 결합하는 것인데, 두 알고리즘의 강점은 상호 보완적이기 때문입니다. 이를 "하이브리드 검색(hybrid search)"라고도 합니다. 스파스 리트리버는 키워드를 기반으로 관련 문서를 찾는 데 능숙하고, 밀도 리트리버는 의미적 유사성을 기반으로 관련 문서를 찾는 데 능숙합니다.


<br><br>

In [13]:
%pip install -qU rank_bm25

Note: you may need to restart the kernel to use updated packages.


In [15]:
# Ensemble Retriever

from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS

doc_list_1 = [
    "비타민A : 당근, 시금치, 감자 등의 주황색과 녹색 채소에서 섭취할 수 있습니다.",
    "비타민B : 전곡물, 콩, 견과류, 육류 등 다양한 식품에서 찾을 수 있습니다.",
    "비타민C : 오렌지, 키위, 딸기, 브로콜리, 피망 등의 과일과 채소에 많이 들어 있습니다.",
    "비타민D : 연어, 참치, 버섯, 우유, 계란 노른자 등에 함유되어 있습니다.",
    "비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.",
]

# BM25와 FAISS 리트리버의 2종류의 리트리버를 초기화 합니다.
bm25_retriever = BM25Retriever.from_texts(
    doc_list_1, metadatas=[{"source": 1}] * len(doc_list_1)
)
bm25_retriever.k = 2

doc_list_2 = [
    "비타민A : 시력과 피부 건강을 지원합니다.",
    "비타민B : 에너지 대사와 신경계 기능을 돕습니다.",
    "비타민C : 면역 체계를 강화하고 콜라겐 생성을 촉진합니다.",
    "비타민D : 뼈 건강과 면역 체계를 지원합니다.",
    "비타민E : 항산화 작용을 통해 세포를 보호합니다.",
]

embedding = CustomEmbedding()
faiss_vectorstore = FAISS.from_texts(
    doc_list_2, embedding, metadatas=[{"source": 2}] * len(doc_list_2)
)

faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

# doc_list_1은 BM25, doc_list_2는 FAISS를 이용하여 구성

# 2개의 리트리버를 결합하여 앙상블 리트리버를 만듭니다. weights는 두 리트리버의 결과에 대한 가중치입니다.
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

In [16]:
# 검색 결과 문서를 가져옵니다.
query = "비타민A 의 효능은?"
ensemble_result = ensemble_retriever.get_relevant_documents(query)
bm25_result = bm25_retriever.get_relevant_documents(query)
faiss_result = faiss_retriever.get_relevant_documents(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]\n", ensemble_result, end="\n\n")
print("[BM25 Retriever]\n", bm25_result, end="\n\n")
print("[FAISS Retriever]\n", faiss_result, end="\n\n")

[Ensemble Retriever]
 [Document(metadata={'source': 1}, page_content='비타민A : 당근, 시금치, 감자 등의 주황색과 녹색 채소에서 섭취할 수 있습니다.'), Document(metadata={'source': 2}, page_content='비타민A : 시력과 피부 건강을 지원합니다.'), Document(metadata={'source': 1}, page_content='비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.'), Document(metadata={'source': 2}, page_content='비타민D : 뼈 건강과 면역 체계를 지원합니다.')]

[BM25 Retriever]
 [Document(metadata={'source': 1}, page_content='비타민A : 당근, 시금치, 감자 등의 주황색과 녹색 채소에서 섭취할 수 있습니다.'), Document(metadata={'source': 1}, page_content='비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.')]

[FAISS Retriever]
 [Document(metadata={'source': 2}, page_content='비타민A : 시력과 피부 건강을 지원합니다.'), Document(metadata={'source': 2}, page_content='비타민D : 뼈 건강과 면역 체계를 지원합니다.')]



In [17]:
# 검색 결과 문서를 가져옵니다.
query = "시력에 좋은 비타민은?"
ensemble_result = ensemble_retriever.get_relevant_documents(query)
bm25_result = bm25_retriever.get_relevant_documents(query)
faiss_result = faiss_retriever.get_relevant_documents(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]\n", ensemble_result, end="\n\n")
print("[BM25 Retriever]\n", bm25_result, end="\n\n")
print("[FAISS Retriever]\n", faiss_result, end="\n\n")

[Ensemble Retriever]
 [Document(metadata={'source': 1}, page_content='비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.'), Document(metadata={'source': 2}, page_content='비타민A : 시력과 피부 건강을 지원합니다.'), Document(metadata={'source': 1}, page_content='비타민D : 연어, 참치, 버섯, 우유, 계란 노른자 등에 함유되어 있습니다.'), Document(metadata={'source': 2}, page_content='비타민D : 뼈 건강과 면역 체계를 지원합니다.')]

[BM25 Retriever]
 [Document(metadata={'source': 1}, page_content='비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.'), Document(metadata={'source': 1}, page_content='비타민D : 연어, 참치, 버섯, 우유, 계란 노른자 등에 함유되어 있습니다.')]

[FAISS Retriever]
 [Document(metadata={'source': 2}, page_content='비타민A : 시력과 피부 건강을 지원합니다.'), Document(metadata={'source': 2}, page_content='비타민D : 뼈 건강과 면역 체계를 지원합니다.')]



In [18]:
# 검색 결과 문서를 가져옵니다.
query = "비타민E 는 어떻게 섭취할 수 있나요?"
ensemble_result = ensemble_retriever.get_relevant_documents(query)
bm25_result = bm25_retriever.get_relevant_documents(query)
faiss_result = faiss_retriever.get_relevant_documents(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]\n", ensemble_result, end="\n\n")
print("[BM25 Retriever]\n", bm25_result, end="\n\n")
print("[FAISS Retriever]\n", faiss_result, end="\n\n")

[Ensemble Retriever]
 [Document(metadata={'source': 1}, page_content='비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.'), Document(metadata={'source': 2}, page_content='비타민E : 항산화 작용을 통해 세포를 보호합니다.'), Document(metadata={'source': 1}, page_content='비타민A : 당근, 시금치, 감자 등의 주황색과 녹색 채소에서 섭취할 수 있습니다.'), Document(metadata={'source': 2}, page_content='비타민D : 뼈 건강과 면역 체계를 지원합니다.')]

[BM25 Retriever]
 [Document(metadata={'source': 1}, page_content='비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.'), Document(metadata={'source': 1}, page_content='비타민A : 당근, 시금치, 감자 등의 주황색과 녹색 채소에서 섭취할 수 있습니다.')]

[FAISS Retriever]
 [Document(metadata={'source': 2}, page_content='비타민E : 항산화 작용을 통해 세포를 보호합니다.'), Document(metadata={'source': 2}, page_content='비타민D : 뼈 건강과 면역 체계를 지원합니다.')]

